# Gerador de Documentação DBT - Conexa

- Time: Engenheira de Dados
- Autor: Anselmo Oliveira
- Código auxiliar utilizado para renomear os arquivos e separar view e tabelas.

In [ ]:
import os

# Define o diretório onde os arquivos estão localizados
directory = '../bronze/'

# Itera sobre os arquivos no diretório
for filename in os.listdir(directory):
    # Define o novo nome para o arquivo
    filename2 = filename.replace(".yaml", "")
    new_name = f"{filename2}_schema.yaml"
    
    # Renomeia o arquivo
    os.rename(os.path.join(directory, filename), os.path.join(directory, new_name))

print("Arquivos renomeados com sucesso!")

In [ ]:
# Bibliotecas
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData
from sqlalchemy import inspect
from trino.auth import BasicAuthentication
import pandas as pd
import shutil
import os
import time
load_dotenv(dotenv_path='./../.env')

In [ ]:
key_groq = os.getenv('GROQ_API3')
key_trino = os.getenv('KEY_TRINO')
key_gpt= os.getenv('OPENAI_API_KEY')
engine_trino = os.getenv('ENGINE_TRINO')
url_engine_postgres = os.getenv('ENGINE_POSTGRES')

In [ ]:
schemas = ['bronze']

In [ ]:
for schema in schemas:
    engine_trino = create_engine(
        f"trino://anselmo.oliveira@conexasaude.com.br:{key_trino}@oliver.conexasaude.com.br:8446/delta/{schema}",
        connect_args={
            "auth": BasicAuthentication("anselmo.oliveira@conexasaude.com.br", key_trino),
            "http_scheme": "https",
            "verify": True,
        }
    )

    engine_postgres = create_engine(url_engine_postgres)

    print('🔄 Criando a engine tabelas e views')

    inspector = inspect(engine_trino)
    tabelas_trino = inspector.get_table_names()
    views_trino = inspector.get_view_names()

    for tabela in tabelas_trino:
        current_path = os.path.join(f'../{schema}/', tabela + '_schema.yaml')
        new_path = os.path.join(f'../{schema}/table/', tabela + '_schema.yaml')

        if os.path.exists(current_path):
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            shutil.move(current_path, new_path)
            print(f"✅ Arquivo movido para {new_path}")
        else:
            print(f"⚠️ Arquivo não encontrado: {current_path}")

    for view in views_trino:
        current_path = os.path.join(f'../{schema}/', view + '_schema.yaml')
        new_path = os.path.join(f'../{schema}/stg/', view + '_schema.yaml')
        
        if os.path.exists(current_path):
            os.makedirs(os.path.dirname(new_path), exist_ok=True)
            shutil.move(current_path, new_path)
            print(f"✅ Arquivo movido para {new_path}")
        else:
            print(f"⚠️ Arquivo não encontrado: {current_path}")

In [ ]:
directory = '../'
for root, dirs, files in os.walk(directory):
    for dir in dirs:
        print(f"  Subpasta: {dir}")
    for file in files:
        if file.endswith('.sql'):
            print(f"  Arquivo SQL: {file}")